In [2]:
import pandas as pd
import os

# 1. 폴더 경로 설정
folder_path = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\1. 실험자료\5. Revision experiment\260103_LCR meter 하이드로젤 전극 측정\data"

# 2. 해당 폴더 내의 모든 파일 목록 가져오기
file_list = os.listdir(folder_path)

print(f"작업을 시작합니다. 대상 폴더: {folder_path}")

for filename in file_list:
    # CSV 파일만 골라내기
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # CSV 불러오기
            df = pd.read_csv(file_path)
            
            # 1, 2열만 남기기 (인덱스 0, 1)
            # 만약 데이터가 2열보다 적은 파일이 섞여 있을 경우를 대비해 슬라이싱 사용
            df_new = df.iloc[:, :2]
            
            # 동일한 경로로 저장 (덮어씌우기)
            # index=False: 행 번호 제외, encoding='utf-8-sig': 한글 및 엑셀 호환용
            df_new.to_csv(file_path, index=False, encoding='utf-8-sig')
            
            print(f"성공: {filename}")
            
        except Exception as e:
            print(f"오류 발생 ({filename}): {e}")

print("모든 작업이 완료되었습니다.")

작업을 시작합니다. 대상 폴더: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\1. 실험자료\5. Revision experiment\260103_LCR meter 하이드로젤 전극 측정\data
성공: Cp-Rp.csv
성공: R-X1.csv
성공: R-X2.csv
성공: R-X3.csv
성공: Z-deg1.csv
성공: Z-deg2.csv
성공: Z-deg3.csv
모든 작업이 완료되었습니다.
